# Conversational Agent with Context Awareness

This notebook explores an experiment in building a conversational agent that **remembers context across turns**. Unlike stateless chatbots that forget past interactions, this agent keeps track of prior messages within each session, making conversations more natural and coherent.  
The design uses:
- **Prompt templates** to structure the interaction.  
- **Message history management** to retain and replay conversation context.  
- **A lightweight session-based store** to isolate multiple conversations.  

The focus here is not on polished production code, but on experimenting with the essential building blocks of agents that can "hold a thought" over time.

### Tech-Stack

- **LangChain**: for chaining prompts and managing conversational flow.  
- **OpenAI GPT (gpt-4o-mini)**: as the underlying language model for response generation.  
- **ChatMessageHistory**: to capture and replay conversation history.  
- **RunnableWithMessageHistory**: for injecting session-based memory into the prompt.  


In [ ]:
pip install -q langchain langchain_experimental openai python-dotenv langchain_openai

#### Importing reqquired libraries

In [ ]:

from langchain_openai import ChatOpenAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import os
from dotenv import load_dotenv
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

#### Loading environment variable 

In [ ]:
load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini", max_tokens=1000, temperature=0)

#### In-memory store for chat history

In [ ]:

store = {}

def get_chat_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

#### Build the prompt-template

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

#### prompt-model chaining

In [ ]:
chain = prompt | llm

#### Wrap the chain with message history

In [ ]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,
    input_messages_key="input",
    history_messages_key="history"
)

#### Example

In [ ]:
session_id = "user_123"


response1 = chain_with_history.invoke(
    {"input": "Hello! How are you?"},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response1.content)

response2 = chain_with_history.invoke(
    {"input": "What was my previous message?"},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response2.content)

## Summary
I think I can improve this concept by:

- ***Persistent Memory / Storage***  
  [e.g., Replace in-memory storage with Redis, Postgres, or MongoDB for durable, scalable sessions.]  

- ***Context Management***  
  [e.g., Introduce summarization memory or vector-backed memory (FAISS, Chroma, Weaviate) to handle long conversations efficiently.]  

- ***Knowledge Augmentation (RAG)***  
  [e.g., Integrate retrieval pipelines with pgVector, Pinecone, or Chroma to ground answers in external documents and databases.]  

- ***Tool Integration***  
  [e.g., Extend the agent with LangChain Agents or LangGraph to perform API calls, DB queries, or calculations.]  

- ***Personalization***  
  [e.g., Track user preferences or profiles to deliver adaptive, customized interactions.]  

- ***Monitoring & Reliability***  
  [e.g., Add observability via LangSmith, Prometheus, or OpenTelemetry; implement rate-limit handling and usage tracking.]  

- ***Error Handling & Recovery***  
  [e.g., Add clarification prompts, fallback responses, and repair strategies for ambiguous or failed outputs.]  

This can transform this simple context-aware agent into a **production-ready, scalable, and enterprise-relevant conversational system**.
